In [1]:
from query.datasets.prelude import *
from query.datasets.ingest import ingest_pose

In [51]:
db = Database()

In [ ]:
#Person.objects.filter(frame__video__path='tvnews/videos/MSNBC_20090703_010000_The_Rachel_Maddow_Show.mp4').delete()
#Face.objects.all().count()
#Video.objects.all().distinct('height').values('height')

with open('/app/paths') as f:
    paths = [s.strip() for s in f.readlines()]
    
tag, _ = Tag.objects.get_or_create(name='pose-test')    
for path in paths:
    video = Video.objects.get(path=path)
    VideoTag(video=video, tag=tag).save()

In [ ]:
Video.objects.filter(videotag__tag__name='pose-test').values('path')

In [7]:
#FaceFeatures.objects.filter(face__person__frame__video__videotag__tag__name='pose-test').count()
#print Face.objects.all().order_by('facefeatures__distto').values('id').query
#PersonTrack.objects.filter(video__path='tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4') \
#    .annotate(duration=Track.duration()).values()

#id = 2043
#FaceFeatures.compute_distances(id)
tracks = list(PersonTrack.objects.filter(video__videotag__tag__name='pose-test') \
#tracks = list(PersonTrack.objects.filter(video__path='tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4') \
    .annotate(c=Subquery(
        Face.objects.filter(person__tracks=OuterRef('pk')) \
        .filter(labeler__name='tinyfaces', facefeatures__distto__isnull=False, facefeatures__distto__lte=1.0) \
        .values('person__tracks')
        .annotate(c=Count('*'))
        .values('c'), models.IntegerField()
        )) \
    .filter(c__gt=0))
len(tracks)

283

In [8]:
frames = defaultdict(set)    
for track in tracks:
    frames[track.video.path] |= set(range(track.min_frame, track.max_frame, 3))

# for path in frames.keys():    
#     if path == 'tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4': continue
#     print(path)
#     #print('Deleting')
#     #Person.objects.filter(frame__video=Video.objects.get(path=path))
#     #Pose.objects.filter(person__frame__video=Video.objects.get(path=path)).delete()
#     print('Ingesting')
#     ingest_pose(
#         Video.objects.get(path=path),
#         db.table(path + '_poses_gather'),
#         sorted(frames[path]))
#     print('Ingested!')

with open('/app/chris-matthews-frames', 'w') as f:
    for path, frames in frames.iteritems():
        print(path)
        f.write('{} {}\n'.format(path, ' '.join([str(s) for s in sorted(frames)])))

tvnews/videos/FOXNEWSW_20130402_040000_Hannity.mp4
tvnews/videos/MSNBCW_20120303_020000_The_Rachel_Maddow_Show.mp4
tvnews/videos/MSNBC_20111109_220000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBC_20100827_060000_The_Rachel_Maddow_Show.mp4
tvnews/videos/MSNBCW_20130404_060000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBCW_20140125_000000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBCW_20150520_230000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBCW_20120924_210000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/FOXNEWS_20100728_010000_Hannity.mp4
tvnews/videos/MSNBC_20100630_210000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBC_20110506_230000_Hardball_With_Chris_Matthews.mp4
tvnews/videos/MSNBCW_20120823_040000_The_Rachel_Maddow_Show.mp4
tvnews/videos/MSNBCW_20130402_080000_The_Rachel_Maddow_Show.mp4
tvnews/videos/MSNBCW_20120412_060000_Hardball_With_Chris_Matthews.mp4


In [3]:
def closest_pose(candidates, target):
    noses = [pose.pose_keypoints()[Pose.Nose] for pose in candidates]
    noses, indices = zip(*[(nose[:2], i) for i, nose in enumerate(noses) if nose[2] > 0])
    dists = np.linalg.norm(np.array(noses) - target, axis=1)
    closest = candidates[indices[np.argmin(dists)]]
    return closest

In [55]:
# Match detected poses with the sparse faces 
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame'))
    for face in faces:
        poses = list(Pose.objects.filter(person__frame=face.person.frame))
        if len(poses) == 0:
            continue
        closest = closest_pose(poses, bbox_midpoint(face))
        closest.person = face.person
        closest.save()

In [56]:
# Fill in tracks with poses
for track in tracks:
    faces = list(Face.objects.filter(person__tracks=track).select_related('person__frame') \
                 .order_by('person__frame__number'))
    for i in range(len(faces) - 1):
        try:
            last_pose = Pose.objects.get(person=faces[i].person)
            for j in range(faces[i].person.frame.number, faces[i+1].person.frame.number, 3):
                cur_poses = Pose.objects.filter(person__frame__video=track.video, person__frame__number=j)
                if len(cur_poses) == 0:
                    continue
                closest = closest_pose(cur_poses, last_pose.pose_keypoints()[Pose.Nose][:2])
                closest.person.tracks.add(track)
                closest.save()
                last_pose = closest
        except Pose.DoesNotExist:
            pass      

In [57]:
for track in tracks:
    print(track.id, track.max_frame-track.min_frame, Person.objects.filter(tracks=track).count())

(3988, 1020, 341)
(3994, 600, 201)


In [4]:
# TODO: need to associate faces with poses
[p1, p2] = Pose.objects.all()[:2]

def pose_dist(p1, p2):
    kp1 = p1.pose_keypoints()
    kp2 = p2.pose_keypoints()
    
    weights = defaultdict(float, {
        Pose.LWrist: 0.4,
        Pose.RWrist: 0.4,
        Pose.Nose: 0.1,        
        Pose.LElbow: 0.05,
        Pose.RElbow: 0.05
    })
    weight_vector = [weights[i] for i in range(Pose.POSE_KEYPOINTS)]
    
    dist = np.linalg.norm(kp2[:,:2] - kp1[:,:2], axis=1)
    weighted_dist = np.array([d * w for d, s, w in zip(dist, kp2[:, 2], weight_vector) if s > 0])
    return np.linalg.norm(weighted_dist)

In [6]:
all_dists = []
for track in tracks:
    poses = list(Pose.objects.filter(person__tracks=track).order_by('person__frame__number'))
    dists = [pose_dist(poses[i], poses[i+1]) for i in range(len(poses) - 1)]
    all_dists.append((track.id, np.mean(dists), track.duration()))

pprint(sorted(all_dists, key=itemgetter(1), reverse=True))

[(544, 0.085587267049609075, 8),
 (526, 0.055704442668377659, 10),
 (558, 0.035203824877924374, 14),
 (432, 0.028847280277080549, 22),
 (491, 0.028283478618785909, 57),
 (510, 0.027027021788168945, 20),
 (522, 0.026710241897755622, 37),
 (502, 0.022655125863687901, 53),
 (525, 0.022558063862747633, 16),
 (490, 0.022357948323882836, 72),
 (435, 0.022315314581304497, 24),
 (487, 0.021614651581525295, 31),
 (478, 0.020917978833957813, 41),
 (596, 0.020014221168435462, 28),
 (438, 0.016960143127957041, 64),
 (640, 0.016895490152096994, 6),
 (625, 0.016696972105883095, 22),
 (592, 0.014646016962719175, 8),
 (500, 0.014513806718985042, 18),
 (436, 0.013565350686883196, 26),
 (515, 0.012427645897858055, 26),
 (470, 0.01176938304234541, 10),
 (600, 0.010864488866605888, 10),
 (481, 0.010818233255223368, 10),
 (643, 0.010761149358219677, 20),
 (483, 0.010726049869230826, 14),
 (585, 0.010419506163391799, 12),
 (602, 0.010132901905334611, 6),
 (532, 0.0095096386942751434, 4),
 (475, 0.0090205058